导入第三方库

In [1]:
import utils
import re,os,shutil,random

import numpy as np
import cv2
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec

import datetime
from scipy.signal import wiener

ModuleNotFoundError: No module named 'cv2'

工具函数

In [3]:
# 获取当前时间年月日时分秒
def get_time():
    now = datetime.datetime.now()
    return now.strftime('%H%M%S')

将图像文件分类  
*在ipynb文件中，文件起始地址为当前文件夹*

In [15]:

file_list = utils.get_file_list('../My_image/original')
file_list1 = {}
mouse_numbers = [3,4,12,51]           # 小鼠编号
tape_stripping_numbers = [0,2,4,6,8]  # 胶带剥离次数
fangda = [20,50]                      # 放大倍数
polarization = [0,90]                 # 是否偏振
shiyan_numbers = [0,1,2]              # 实验组号

for file in file_list:
    filename = re.split('_\d_\d\.jpg', file)
    file_list1[filename[0]] = file_list1[filename[0]] + 1 if filename[0] in file_list1 else 0

for i in mouse_numbers:
    for j in polarization:
        for k in fangda:
            # 新建文件夹
            new_path = '../My_image/group_tape_stripping_numbers/' + str(i) + '_' + str(k) + '_' + str(j)
            if not os.path.exists(new_path):
                os.makedirs(new_path)
            for l in tape_stripping_numbers:
                # 随机抽取文件
                shutil.copy('../My_image/original/'+ str(i) + '_' + str(l) + '_' + str(k) + '_' + str(j)+'_3_'+str(random.choice(shiyan_numbers))+'.jpg', \
                    new_path + '/' + str(i) + '_' + str(l) + '_' + str(k) + '_' + str(j) +'.jpg')

转为灰度图像

In [ ]:
file_list = utils.get_file_list('../My_image/group_tape_stripping_numbers/3_20_0')
for file in file_list:
    print(file)
    img = cv2.imread('../My_image/group_tape_stripping_numbers/3_20_0/'+file)
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    plt.hist(gray.ravel(),256,[0,256]) # ravel()函数将多维数组降为一维
    plt.show()

cv2.waitKey(0)

得到ROI的区域坐标

In [4]:
# img = cv2.imread("My_image/group_tape_stripping_numbers/3_20_0/3_0_20_0.jpg")
# roi = cv2.selectROI(windowName="roi", img=img, showCrosshair=True, fromCenter=False)
x, y, w, h = (119,130,263,133)
img = cv2.imread('../My_image/group_tape_stripping_numbers/3_20_0/3_0_20_0.jpg')
# 获取矩阵最大值

print(img.min())
# cv2.waitKey(0)
# cv2.destroyAllWindows()

42


提取ROI

In [ ]:
file_list = utils.get_file_list('../My_image/group_tape_stripping_numbers/4_20_0')
for file in file_list:
    img = cv2.imread('../My_image/group_tape_stripping_numbers/4_20_0/'+file)
    imgROI = img[y:y+h, x:x+w].copy()
    grayROI = cv2.cvtColor(imgROI, cv2.COLOR_BGR2GRAY)
    plt.hist(grayROI.ravel(),256,[0,256])
    plt.show()
cv2.waitKey(0)

求三通道直方图以及直方图均值  
**cv2读取是BGR，plt读取是RGB**

In [6]:
# 计算三个通道的直方图
def hist(image,ax):
    color = ('blue','green','red')
    for i,color in enumerate(color):
        hist = cv2.calcHist([image],[i],None,[256],[0,256])
        ax.plot(hist,color=color)
        ax.set_xlim([0,256])

def hist_mean(image):
    color = ('blue','green','red')
    mean = []
    for i,color in enumerate(color):
        hist = cv2.calcHist([image],[i],None,[256],[0,256])
        sum = 0
        for j,hist1 in enumerate(hist):
            sum = sum + (j+1)*hist1
        mean.append(round(sum[0]/image.shape[0]/image.shape[1],2))
    return mean

计算五张图片的提取ROI均值滤波直方图(合并在一张图上)

In [ ]:

file_path = '../My_image/group_tape_stripping_numbers/51_20_90'
file_list = utils.get_file_list(file_path)
figure = plt.figure(figsize=(30,30))   # 创建画布
gs = gridspec.GridSpec(5,4)          # 创建子图
mean_list_before = []
mean_list_after = []
i = 0
for file in file_list:
    img = cv2.imread(file_path+'/'+file)
    rgb = img[:,:,::-1]    # BGR转RGB
    plt.rcParams['font.sans-serif'] = ['SimHei']    # 用来正常显示中文标签
    
    ax1 = figure.add_subplot(gs[i,0],title = file)
    ax1.axis('off')
    ax1.imshow(rgb)

    ax2 = figure.add_subplot(gs[i,1],title = '原图对应直方图')
    hist(img,ax2)
    mean = hist_mean(img)
    mean_list_before.append(mean)
    # print(mean)
    ax2.text(0, 6000, 'b通道均值：'+str(mean[0]), fontsize=13)
    ax2.text(0, 5000, 'g通道均值：'+str(mean[1]), fontsize=13)
    ax2.text(0, 4000, 'r通道均值：'+str(mean[2]), fontsize=13)

    ax3 = figure.add_subplot(gs[i,2],title = '提取ROI并均值滤波')  
    imgROI = img[y:y+h, x:x+w].copy()
    junzhi_result = cv2.blur(imgROI, (5, 5))
    # grayROI = cv2.cvtColor(cv2.blur(imgROI, (5, 5)), cv2.COLOR_BGR2GRAY)
    ax3.axis('off')
    ax3.imshow(junzhi_result[:,:,::-1])

    ax4 = figure.add_subplot(gs[i,3],title = 'ROI对应直方图')
    hist(junzhi_result,ax4)
    mean = hist_mean(junzhi_result)
    mean_list_after.append(mean)
    # print(mean)
    ax4.text(0, 1000, 'b通道均值：'+str(mean[0]), fontsize=13)
    ax4.text(0, 750, 'g通道均值：'+str(mean[1]), fontsize=13)
    ax4.text(0, 500, 'r通道均值：'+str(mean[2]), fontsize=13)
    i += 1
# plot保存图片

plt.savefig('./Result_image/'+re.split('/', file_path)[-1]+str(get_time())+'junzhilvbo.jpg')
plt.show()

colors = ('blue','green','red')
r = []
g = []
b = []
for i,means in enumerate(mean_list_before):
    r.append(means[2])
    g.append(means[1])
    b.append(means[0])

print(r,g,b)
plt.rcParams['font.sans-serif'] = ['SimHei']    # 用来正常显示中文标签
plt.rcParams['axes.unicode_minus'] = False      # 用来正常显示负号
plt.figure(figsize=(20,10))
# 设置标题
plt.title('均值滤波前的直方图均值')

plt.plot(r, color='red', label='r')
plt.plot(g, color='green', label='g')
plt.plot(b, color='blue', label='b')
plt.legend()
plt.show()

r1 = []
g1 = []
b1 = []
for i,means in enumerate(mean_list_after):
    r1.append(means[2])
    g1.append(means[1])
    b1.append(means[0])

print(r1,g1,b1)
plt.figure(figsize=(20,10))
# 设置标题
plt.title('均值滤波后的直方图均值')

plt.plot(r1, color='red', label='r')
plt.plot(g1, color='green', label='g')
plt.plot(b1, color='blue', label='b')
plt.legend()
plt.show()


cv2.waitKey(0)
cv2.destroyAllWindows()

计算五张图片的提取ROI方框滤波直方图(合并在一张图上)

In [ ]:
file_path = '../My_image/group_tape_stripping_numbers/51_20_90'
file_list = utils.get_file_list(file_path)
figure = plt.figure(figsize=(30,30))   # 创建画布
gs = gridspec.GridSpec(5,4)          # 创建子图
mean_list_before = []
mean_list_after = []
i = 0
for file in file_list:
    img = cv2.imread(file_path+'/'+file)
    rgb = img[:,:,::-1]    # BGR转RGB
    plt.rcParams['font.sans-serif'] = ['SimHei']    # 用来正常显示中文标签
    
    ax1 = figure.add_subplot(gs[i,0],title = file)
    ax1.axis('off')
    ax1.imshow(rgb)

    ax2 = figure.add_subplot(gs[i,1],title = '原图对应直方图')
    hist(img,ax2)
    mean = hist_mean(img)
    mean_list_before.append(mean)
    # print(mean)
    ax2.text(0, 3000, 'b通道均值：'+str(mean[0]), fontsize=13)
    ax2.text(0, 2000, 'g通道均值：'+str(mean[1]), fontsize=13)
    ax2.text(0, 1000, 'r通道均值：'+str(mean[2]), fontsize=13)

    ax3 = figure.add_subplot(gs[i,2],title = '提取ROI并方框滤波')  
    imgROI = img[y:y+h, x:x+w].copy()
    fangkuang_result = cv2.boxFilter(imgROI, -1, (3, 3), normalize=True)
    # grayROI = cv2.cvtColor(cv2.blur(imgROI, (5, 5)), cv2.COLOR_BGR2GRAY)
    ax3.axis('off')
    ax3.imshow(fangkuang_result[:,:,::-1])

    ax4 = figure.add_subplot(gs[i,3],title = 'ROI对应直方图')
    hist(fangkuang_result,ax4)
    mean = hist_mean(fangkuang_result)
    mean_list_after.append(mean)
    # print(mean)
    ax4.text(0, 1000, 'b通道均值：'+str(mean[0]), fontsize=13)
    ax4.text(0, 750, 'g通道均值：'+str(mean[1]), fontsize=13)
    ax4.text(0, 500, 'r通道均值：'+str(mean[2]), fontsize=13)
    i += 1
# plot保存图片

plt.savefig('./Result_image/'+re.split('/', file_path)[-1]+str(get_time())+'fangkuanglvbo.jpg')
plt.show()

colors = ('blue','green','red')
r = []
g = []
b = []
for i,means in enumerate(mean_list_before):
    r.append(means[2])
    g.append(means[1])
    b.append(means[0])

print(r,g,b)
plt.rcParams['font.sans-serif'] = ['SimHei']    # 用来正常显示中文标签
plt.rcParams['axes.unicode_minus'] = False      # 用来正常显示负号
plt.figure(figsize=(20,10))
# 设置标题
plt.title('方框滤波前的直方图均值')

plt.plot(r, color='red', label='r')
plt.plot(g, color='green', label='g')
plt.plot(b, color='blue', label='b')
plt.legend()
plt.show()

r1 = []
g1 = []
b1 = []
for i,means in enumerate(mean_list_after):
    r1.append(means[2])
    g1.append(means[1])
    b1.append(means[0])

print(r1,g1,b1)
plt.figure(figsize=(20,10))
# 设置标题
plt.title('方框滤波后的直方图均值')

plt.plot(r1, color='red', label='r')
plt.plot(g1, color='green', label='g')
plt.plot(b1, color='blue', label='b')
plt.legend()
plt.show()



cv2.waitKey(0)
cv2.destroyAllWindows()

计算五张图片的提取ROI高斯滤波直方图(合并在一张图上)

In [1]:

file_path = '../My_image/group_tape_stripping_numbers/4_20_90'
file_list = utils.get_file_list(file_path)
figure = plt.figure(figsize=(30,30))   # 创建画布
gs = gridspec.GridSpec(5,4)          # 创建子图
mean_list_after = []
mean_list_before = []
i = 0
for file in file_list:
    img = cv2.imread(file_path+'/'+file)
    rgb = img[:,:,::-1]    # BGR转RGB
    plt.rcParams['font.sans-serif'] = ['SimHei']    # 用来正常显示中文标签
    
    ax1 = figure.add_subplot(gs[i,0],title = file)
    ax1.axis('off')
    ax1.imshow(rgb)

    ax2 = figure.add_subplot(gs[i,1],title = '原图对应直方图')
    hist(img,ax2)
    mean = hist_mean(img)
    mean_list_before.append(mean)

    ax2.text(0, 4000, 'b通道均值：'+str(mean[0]), fontsize=13)
    ax2.text(0, 3000, 'g通道均值：'+str(mean[1]), fontsize=13)
    ax2.text(0, 2000, 'r通道均值：'+str(mean[2]), fontsize=13)

    ax3 = figure.add_subplot(gs[i,2],title = '提取ROI并均值滤波')  
    imgROI = img[y:y+h, x:x+w].copy()
    Gauss_result = cv2.GaussianBlur(imgROI, (5, 5), 0)
    # grayROI = cv2.cvtColor(cv2.blur(imgROI, (5, 5)), cv2.COLOR_BGR2GRAY)
    ax3.axis('off')
    ax3.imshow(Gauss_result[:,:,::-1])

    ax4 = figure.add_subplot(gs[i,3],title = 'ROI对应直方图')
    hist(Gauss_result,ax4)
    mean = hist_mean(Gauss_result)
    mean_list_after.append(mean)
    # print(mean)
    ax4.text(0, 1000, 'b通道均值：'+str(mean[0]), fontsize=13)
    ax4.text(0, 750, 'g通道均值：'+str(mean[1]), fontsize=13)
    ax4.text(0, 500, 'r通道均值：'+str(mean[2]), fontsize=13)
    i += 1
# plot保存图片

# plt.savefig('./Result_image/'+re.split('/', file_path)[-1]+str(get_time())+'Gausslvbo.jpg')
plt.show()

colors = ('blue','green','red')
r = []
g = []
b = []
for i,means in enumerate(mean_list_before):
    r.append(means[2])
    g.append(means[1])
    b.append(means[0])

print(r,g,b)
plt.rcParams['font.sans-serif'] = ['SimHei']    # 用来正常显示中文标签
plt.rcParams['axes.unicode_minus'] = False      # 用来正常显示负号
plt.figure(figsize=(20,10))
# 设置标题
plt.title('高斯滤波前的直方图均值')

plt.plot(r, color='red', label='r')
plt.plot(g, color='green', label='g')
plt.plot(b, color='blue', label='b')
plt.legend()
plt.show()

r1 = []
g1 = []
b1 = []
for i,means in enumerate(mean_list_after):
    r1.append(means[2])
    g1.append(means[1])
    b1.append(means[0])

print(r,g,b)
plt.figure(figsize=(20,10))
# 设置标题
plt.title('高斯滤波后的直方图均值')

plt.plot(r, color='red', label='r')
plt.plot(g, color='green', label='g')
plt.plot(b, color='blue', label='b')
plt.legend()
plt.show()


cv2.waitKey(0)
cv2.destroyAllWindows()

NameError: name 'utils' is not defined

计算五张图片的提取ROI维纳滤波直方图(合并在一张图上)

In [ ]:

file_path = '../My_image/group_tape_stripping_numbers/51_20_0'
file_list = utils.get_file_list(file_path)
figure = plt.figure(figsize=(30,30))   # 创建画布
gs = gridspec.GridSpec(5,4)          # 创建子图
mean_list_after = []
mean_list_before = []
i = 0
for file in file_list:
    img = cv2.imread(file_path+'/'+file)
    rgb = img[:,:,::-1]    # BGR转RGB
    plt.rcParams['font.sans-serif'] = ['SimHei']    # 用来正常显示中文标签
    
    ax1 = figure.add_subplot(gs[i,0],title = file)
    ax1.axis('off')
    ax1.imshow(rgb)

    ax2 = figure.add_subplot(gs[i,1],title = '原图对应直方图')
    hist(img,ax2)
    mean = hist_mean(img)
    mean_list_before.append(mean)

    ax2.text(0, 4000, 'b通道均值：'+str(mean[0]), fontsize=13)
    ax2.text(0, 3000, 'g通道均值：'+str(mean[1]), fontsize=13)
    ax2.text(0, 2000, 'r通道均值：'+str(mean[2]), fontsize=13)

    ax3 = figure.add_subplot(gs[i,2],title = '提取ROI并均值滤波')  
    imgROI = img[y:y+h, x:x+w].copy()
    Gauss_result = cv2.GaussianBlur(imgROI, (5, 5), 0)
    # grayROI = cv2.cvtColor(cv2.blur(imgROI, (5, 5)), cv2.COLOR_BGR2GRAY)
    ax3.axis('off')
    ax3.imshow(Gauss_result[:,:,::-1])

    ax4 = figure.add_subplot(gs[i,3],title = 'ROI对应直方图')
    hist(Gauss_result,ax4)
    mean = hist_mean(Gauss_result)
    mean_list_after.append(mean)
    # print(mean)
    ax4.text(0, 1000, 'b通道均值：'+str(mean[0]), fontsize=13)
    ax4.text(0, 750, 'g通道均值：'+str(mean[1]), fontsize=13)
    ax4.text(0, 500, 'r通道均值：'+str(mean[2]), fontsize=13)
    i += 1
# plot保存图片

# plt.savefig('./Result_image/'+re.split('/', file_path)[-1]+str(get_time())+'Gausslvbo.jpg')
plt.show()

colors = ('blue','green','red')
r = []
g = []
b = []
for i,means in enumerate(mean_list_before):
    r.append(means[2])
    g.append(means[1])
    b.append(means[0])

print(r,g,b)
plt.rcParams['font.sans-serif'] = ['SimHei']    # 用来正常显示中文标签
plt.rcParams['axes.unicode_minus'] = False      # 用来正常显示负号
plt.figure(figsize=(20,10))
# 设置标题
plt.title('高斯滤波前的直方图均值')

plt.plot(r, color='red', label='r')
plt.plot(g, color='green', label='g')
plt.plot(b, color='blue', label='b')
plt.legend()
plt.show()

r1 = []
g1 = []
b1 = []
for i,means in enumerate(mean_list_before):
    r1.append(means[2])
    g1.append(means[1])
    b1.append(means[0])

print(r,g,b)
plt.figure(figsize=(20,10))
# 设置标题
plt.title('高斯滤波后的直方图均值')

plt.plot(r, color='red', label='r')
plt.plot(g, color='green', label='g')
plt.plot(b, color='blue', label='b')
plt.legend()
plt.show()


cv2.waitKey(0)
cv2.destroyAllWindows()

灰度共生矩阵和纹理特征

In [ ]:

#  """计算灰度共生矩阵"""
def fast_glcm(img, vmin=0, vmax=255, levels=8, kernel_size=5, distance=1.0, angle=0.0):
    # '''
    # Parameters
    # ----------
    # img: array_like, shape=(h,w), dtype=np.uint8
    #     input image
    # vmin: int
    #     minimum value of input image
    # vmax: int
    #     maximum value of input image
    # levels: int
    #     number of grey-levels of GLCM
    # kernel_size: int
    #     Patch size to calculate GLCM around the target pixel
    # distance: float
    #     pixel pair distance offsets [pixel] (1.0, 2.0, and etc.)
    # angle: float
    #     pixel pair angles [degree] (0.0, 30.0, 45.0, 90.0, and etc.)
    # Returns
    # -------
    # Grey-level co-occurrence matrix for each pixels
    # shape = (levels, levels, h, w)
    # '''
    
    # digitize
    bins = np.linspace(vmin, vmax+1, levels+1)
    gl1 = np.digitize(img, bins) - 1
 
    # make shifted image
    dx = distance*np.cos(np.deg2rad(angle))
    dy = distance*np.sin(np.deg2rad(-angle))
    mat = np.array([[1.0,0.0,-dx], [0.0,1.0,-dy]], dtype=np.float32)
    h,w = img.shape
    gl2 = cv2.warpAffine(gl1, mat, (w,h), flags=cv2.INTER_NEAREST,
                         borderMode=cv2.BORDER_REPLICATE)
 
    # make glcm
    glcm = np.zeros((levels, levels, h, w), dtype=np.uint8)
    for i in range(levels):
        for j in range(levels):
            mask = ((gl1==i) & (gl2==j))
            glcm[i,j, mask] = 1
 
    kernel = np.ones((kernel_size, kernel_size), dtype=np.uint8)
    for i in range(levels):
        for j in range(levels):
            glcm[i,j] = cv2.filter2D(glcm[i,j], -1, kernel)
 
    glcm = glcm.astype(np.float32)
    return glcm
 
# """计算纹理特征"""
def fast_textural_features(glcm, vmin=0, vmax=255, levels=8, kernel_size=5):
    _,_,h,w = glcm.shape
    # 计算最大值
    max_  = np.max(glcm, axis=(0,1))
    # 计算熵
    pnorm = glcm / np.sum(glcm, axis=(0,1)) + 1./kernel_size**2
    ent  = np.sum(-pnorm * np.log(pnorm), axis=(0,1))
    # 均值/标准差/对比度/相异性/同质性/角二阶距
    mean,std2,cont,diss,homo,asm = [np.zeros((h,w), dtype=np.float32) for x in range(6)]
    for i in range(levels):
        for j in range(levels):
            mean += glcm[i,j] * i / (levels)**2 # 计算均值
            cont += glcm[i,j] * (i-j)**2 # 计算对比度
            diss += glcm[i,j] * np.abs(i-j) # 计算相异性
            homo += glcm[i,j] / (1.+(i-j)**2) # 计算同质性
            asm  += glcm[i,j]**2 # 计算角二阶距
    # 计算能量
    ene = np.sqrt(asm)
    # 计算标准差
    for i in range(levels):
        for j in range(levels):
            std2 += (glcm[i,j] * i - mean)**2
    std = np.sqrt(std2)   
    
    return [max_,ent,mean,std,cont,diss,homo,asm,ene]
 
 
if __name__ == '__main__':
    
    img = cv2.imread("../My_image/group_tape_stripping_numbers/4_20_0/4_0_20_0.jpg",0) # 以灰度读取
    glcm = fast_glcm(img, vmin=0, vmax=255, levels=8, kernel_size=5, distance=1.0, angle=0.0) # 计算共生矩阵
    textural = fast_textural_features(glcm, vmin=0, vmax=255, levels=8, kernel_size=5)
    print(textural)
 
 

sobel算子特征提取

In [18]:

# Load the image
img = cv2.imread("../My_image/111.jpg", cv2.IMREAD_GRAYSCALE)

# Apply the Sobel filter
sobelx = cv2.Sobel(img, cv2.CV_64F, 1, 0, ksize=3)
sobely = cv2.Sobel(img, cv2.CV_64F, 0, 1, ksize=3)

# Combine the results to get the final image
sobel = cv2.addWeighted(sobelx, 0.5, sobely, 0.5, 0)

# Show the image
cv2.imshow("Sobel", sobel)
cv2.waitKey(0)
cv2.destroyAllWindows()


维纳滤波

In [12]:


# Load the image
img = cv2.imread("../My_image/group_tape_stripping_numbers/4_20_0/4_0_20_0.jpg", cv2.IMREAD_GRAYSCALE)

# Apply Wiener filter
wiener_filtered = wiener(img, (3, 3))

# Show the filtered image
cv2.imshow("Wiener filtered", wiener_filtered)
cv2.waitKey(0)
cv2.destroyAllWindows()


直方图均衡化

In [17]:

# Load the image
img = cv2.imread("image.jpg", cv2.IMREAD_GRAYSCALE)

# Perform histogram equalization
equalized = cv2.equalizeHist(img)

# Show the original and equalized images
cv2.imshow("Original", img)
cv2.imshow("Equalized", equalized)
cv2.waitKey(0)
cv2.destroyAllWindows()



测试用例

In [19]:
import cv2
import numpy as np
from scipy.signal import wiener

# Load the image
img = cv2.imread("../My_image/group_tape_stripping_numbers/4_20_0/4_0_20_0.jpg", cv2.IMREAD_GRAYSCALE)

# Define the kernel size and standard deviation for Gaussian filter
gaussian_kernel_size = (5, 5)
gaussian_sigma = 1

# Create the kernel and apply the Gaussian filter
gaussian_kernel = np.ones(gaussian_kernel_size, np.float32) / (gaussian_kernel_size[0] * gaussian_kernel_size[1])
gaussian_filtered = cv2.filter2D(img, -1, gaussian_kernel)

# Define the kernel size for Wiener filter
wiener_kernel_size = (5, 5)

# Apply the Wiener filter
wiener_filtered = wiener(gaussian_filtered, (wiener_kernel_size))

# Show the filtered image
cv2.imshow("Wiener filtered after Gaussian filtered", wiener_filtered)
cv2.waitKey(0)
cv2.destroyAllWindows()

